In [ ]:
# !/usr/bin/env python3
#coding=utf-8

import binascii
import socket
import struct
import sys
from Arm_Lib import Arm_Device
import select
import torch
import torchvision
import cv2
import ipywidgets.widgets as widgets
import threading
from time import time
import enum
import colorsys
import numpy as np


# Fix Address already in use
import os
os.system('fuser -kn tcp 65432 | awk')


image_width = 352
image_height = 288
cam = cv2.VideoCapture(0)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, image_width)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, image_height)
cam.set(cv2.CAP_PROP_FPS, 30)
    
print("torch cuda is available:", torch.cuda.is_available())
model = torch.load("cube_model_500.pth", map_location=torch.device('cuda'))

image_widget = widgets.Image(format='jpeg', width=512, height=512)  #Set up camera display components
display(image_widget)      


def bgr8_to_jpeg(value, quality=75):
    if value is None:
        return None
    return bytes(cv2.imencode('.jpg', value)[1])  


def random_colours(N, enable_random=True, num_channels=3):
    """
    Generate random colors.
    Generate visually distinct colours by linearly spacing the hue 
    channel in HSV space and then convert to RGB space.
    """
    start = 0
    if enable_random:
        random.seed(10)
        start = random.random()
    hues = [(start + i / N) % 1.0 for i in range(N)]
    colours = [list(colorsys.hsv_to_rgb(h, 0.9, 1.0)) for i, h in enumerate(hues)]
    if num_channels == 4:
        for color in colours:
            color.append(1.0)
    if enable_random:
        random.shuffle(colours)
    return colours


def draw_boxes(image, bboxes, labels=None, colours=None, label_size=10):
    if colours is None:
        colours = random_colours(len(bboxes))
    if labels is None:
        labels = [""] * len(bboxes)
    for bb, label, colour in zip(bboxes, labels, colours):
        maxint = 2 ** (struct.Struct("i").size * 8 - 1) - 1
        # if a bbox is not visible, do not draw
        if bb[0] != maxint and bb[1] != maxint:
            start_point = (int(bb[0]),int(bb[3]))
            end_point = (int(bb[2]), int(bb[1]))
            color = (0,255,0)
            
            image = cv2.rectangle(image, start_point, end_point, color, 3)

    # show the image        
    image_widget.value = bgr8_to_jpeg(image)
    

def run_inferencing(model, image):
    torch_image = torch.tensor(image, dtype=torch.float, device="cuda")
    torch_image = torch_image.float() / 255.0
    torch_image = torch_image.permute(2, 0, 1)
    
    model.eval()
    with torch.no_grad():
        
        predictions = model([torch_image])
        
        #print(predictions)
               
        idx = 0
        score_thresh = 0.8

        pred = predictions[idx]

        score_filter = [i for i in range(len(pred["scores"])) if pred["scores"][i] > score_thresh]
        num_instances = len(score_filter)
        colours = random_colours(num_instances, enable_random=False)
        
        categories = ["None", "Cube", "Sphere", "Cone"]
        mapping = {i + 1: cat for i, cat in enumerate(categories)}
        labels = [mapping[label.item()] for label in pred["labels"]]

        draw_boxes(image, pred["boxes"].tolist(), labels=labels, colours=colours, label_size=10)
    

def read_camera_and_inference():
    while True:
        ret, frame = cam.read()
        if frame is not None :
            run_inferencing(model, frame)
      
        
read_camera_and_inference()

In [ ]:
del Arm  # Release DOFBOT object